In [53]:
# Aqui é feito a importação dos dados ncbi_saqua, separados os ids de embl e refseq.

import pandas as pd
import numpy as np
import re


def create_chunks(list_name, n):
    for i in range(0, len(list_name), n):
        yield list_name[i:i + n]

df_saqua_ncbi = pd.read_table("C:/Users/eulle/Documents/besthits_ncbi_saqua.m8")

columns_csv = ['qseqid' ,'sseqid' ,'pident' ,'length','mismatch','gapopen','qstart' ,'qend' ,'sstart' ,'send' ,'evalue' ,'bitscore']
row_values = pd.Series(df_saqua_ncbi.columns)
df_saqua_ncbi = pd.DataFrame(np.vstack([df_saqua_ncbi.columns,df_saqua_ncbi]))
df_saqua_ncbi.columns = columns_csv

list_genkbank = list(df_saqua_ncbi["sseqid"])
list_genkbank = list(set(list_genkbank))
p_regex = re.compile(r'^WP')
refseq_id = [ii for ii in list_genkbank if p_regex.match(ii)]
embl_id = [ii for ii in list_genkbank if not p_regex.match(ii)]   


In [2]:
# Aqui são todas as bibliotecas e func utilizadas
# Ter cuidado com o mapping_ids para quando for rodar n esquecer de mudar os parâmetros
import pandas as pd
import numpy as np
import re,glob,os
import urllib.parse
import urllib.request
from multiprocessing.pool import ThreadPool
import urllib
import pandas as pd
import operator
import functools
from operator import methodcaller

URL = 'https://www.uniprot.org/uploadlists/'

def create_chunks(list_name, n):
    for i in range(0, len(list_name), n):
        yield list_name[i:i + n]
        
def mapping_ids(ids2map, source_fmt='ACC',target_fmt='KO_ID', output_fmt='tab'):
    if hasattr(ids2map, 'pop'):
            ids2map = ' '.join(ids2map)
    params = {
    'from': source_fmt,
    'to': target_fmt,
    'format': output_fmt,
    'query': ids2map
    }

    data = urllib.parse.urlencode(params)
    data = data.encode('utf-8')
    req = urllib.request.Request(URL, data)
    with urllib.request.urlopen(req) as f:
        response = f.read().decode('utf-8')
    return response

def concat_dfs(path,extension):
    extension = extension
    all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
    
    combined_dfs = pd.concat([pd.read_table(f) for f in all_filenames ])
    
    return combined_dfs


In [10]:
# Aqui é feito o mapeamento de refseq e embl para uniprot

import urllib.parse
import urllib.request
from multiprocessing.pool import ThreadPool
import glob,os


URL = 'https://www.uniprot.org/uploadlists/'

def concat_dfs(path,extension):
    extension = extension
    all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
    
    combined_dfs = pd.concat([pd.read_table(f) for f in all_filenames ])
    
    return combined_dfs


def create_chunks(list_name, n):
    for i in range(0, len(list_name), n):
        yield list_name[i:i + n]

def mapping_ids(ids2map, source_fmt='P_REFSEQ_AC',target_fmt='ACC', output_fmt='tab'):
    if hasattr(ids2map, 'pop'):
            ids2map = ' '.join(ids2map)
    params = {
    'from': source_fmt,
    'to': target_fmt,
    'format': output_fmt,
    'query': ids2map
    }

    data = urllib.parse.urlencode(params)
    data = data.encode('utf-8')
    req = urllib.request.Request(URL, data)
    with urllib.request.urlopen(req) as f:
        response = f.read().decode('utf-8')
    return response

list_urls = []

chunk_embl = list(create_chunks(embl_id,1000))
chunk_refseq = list(create_chunks(refseq_id,1000))

aux = ThreadPool(10).imap(mapping_ids,chunk_refseq)

for index,ii in enumerate(aux): 
    with open('genkbank_ids/'+ str(index)+'_refseq'+'.csv', 'w') as file:
        file.write(ii)

In [23]:
# Mapeando os embl_ids para ko

concat_embl = concat_dfs("genkbank_ids/embl/","csv")
# 274 rows duplicated -- EMBL
concat_embl = concat_embl.reset_index(drop=True)
concat_embl.columns = ["EMBL","Uniprot_ID"]
chunk_uniprotid = list(create_chunks(list(concat_embl["Uniprot_ID"]),1000))

pool_uniprot_ko_embl = ThreadPool(10).imap(mapping_ids,chunk_uniprotid)

for index,ii in enumerate(pool_uniprot_ko_embl): 
    with open('uniprot_ko/'+ str(index)+'_ko'+'.csv', 'w') as file:
        file.write(ii)
#concat_embl.loc[concat_embl["EMBL"].duplicated()]

concat_embl_ko = concat_dfs("genkbank_ids/embl/uniprot_ko/","csv")
concat_embl_ko = concat_embl_ko.reset_index(drop=True)
concat_embl_ko.columns = ["Uniprot_ID","KO"]

In [67]:
# Mapeando os refseq_ids para ko
# lembrar de mudar esse concat

concat_refseq = concat_dfs(os.chdir("./"),"csv")

concat_refseq = concat_refseq.reset_index(drop=True)
concat_refseq.columns = ["REFSEQ","Uniprot_ID"]
chunk_uniprotid_refseq = list(create_chunks(list(concat_refseq["Uniprot_ID"]),1000))

pool_uniprot_ko_refseq = ThreadPool(10).imap(mapping_ids,chunk_uniprotid_refseq)

for index,ii in enumerate(pool_uniprot_ko_refseq): 
    with open('uniprot_ko_refseq/'+ str(index)+'_ko_refseq'+'.csv', 'w') as file:
        file.write(ii)
# Da próxima vez que vc entrar só precisa ler essas linhas de baixo
concat_refseq_ko = concat_dfs(os.chdir("uniprot_ko_refseq/"),"csv")
concat_refseq_ko = concat_refseq_ko.reset_index(drop=True)
concat_refseq_ko.columns = ["Uniprot_ID","KO"]

concat_refseq_ko.to_csv("../uniprot_ko_refseq/df_ko_refseq.csv",sep="\t",encoding="utf-8",index=False)
concat_embl_ko.to_csv("../../embl/uniprot_ko/df_ko_embl.csv",sep="\t",encoding="utf-8",index=False)

## meter esses dois dfs no rstudio para mapear as vias


FileNotFoundError: [WinError 2] O sistema não pode encontrar o arquivo especificado: '../../refseq/'

In [1]:
# Concatenação dos dados

import pandas as pd
df_genkbank = pd.read_csv("pathways_ncbixsaqua")

concat_embl = concat_dfs(os.chdir("./"),"csv")
# 274 rows duplicated -- EMBL
concat_embl = concat_embl.reset_index(drop=True)
concat_embl.columns = ["genkbank","Uniprot_ID"]

concat_refseq = concat_dfs(os.chdir("../refseq/"),"csv")

concat_refseq = concat_refseq.reset_index(drop=True)
concat_refseq.columns = ["genkbank","Uniprot_ID"]

bigdata = pd.concat([concat_refseq,concat_embl])
df_geral = pd.merge(bigdata,df_genkbank,on="Uniprot_ID",how="left")

teta = df_geral[df_geral["Pathway"].notna()]
teta = teta.reset_index()
teta = teta.drop_duplicated("genkbank")


In [47]:


df_saqua_ncbi = pd.merge(df_saqua_ncbi,teta,left_on="sseqid",right_on="genkbank",how="left")
df_saqua_ncbi.to_csv("../../saqua_ncbi_final.csv",sep="\t",encoding="utf-8",index=False)

df_saqua_ncbi = df_saqua_ncbi[df_saqua_ncbi["Pathway"].notna()] # 25497
df_saqua_ncbi = df_saqua_ncbi.reset_index()
df_saqua_ncbi = df_saqua_ncbi.drop(columns=["level_0","index"])

#df_saqua_ncbi[df_saqua_ncbi["Pathway"].notna()]

KeyError: 'Pathway'

In [3]:
df_saqua_ncbi = pd.read_table("saqua_ncbi_final.csv")
df_saqua_ncbi = df_saqua_ncbi[df_saqua_ncbi["Pathway"].notna()]
pathways = df_saqua_ncbi["Pathway"]
pathways = list(pathways)
pathways = map(methodcaller("split",";"),pathways)
pathways = functools.reduce(operator.iconcat, pathways, [])
pathways = list(set(pathways))
pathways
# 221 vias distintas

['GABAergic synapse',
 'Propanoate metabolism',
 'Longevity regulating pathway',
 'Central carbon metabolism in cancer',
 'Huntington disease',
 'Insulin signaling pathway',
 'Ether lipid metabolism',
 'Plant-pathogen interaction',
 'Base excision repair',
 'Riboflavin metabolism',
 'Adipocytokine signaling pathway',
 'Biosynthesis of secondary metabolites',
 'HIF-1 signaling pathway',
 'One carbon pool by folate',
 'Nucleotide excision repair',
 'Glucagon signaling pathway',
 'Citrate cycle (TCA cycle)',
 'Flagellar assembly',
 'Viral carcinogenesis',
 'Photosynthesis',
 'RNA degradation',
 'Proteoglycans in cancer',
 'Necroptosis',
 'Carbon metabolism',
 'Cushing syndrome',
 'Lipoic acid metabolism',
 'Renal cell carcinoma',
 'Arginine and proline metabolism',
 'Parkinson disease',
 'Axon regeneration',
 'Aminobenzoate degradation',
 'Type II diabetes mellitus',
 'Pathways in cancer',
 'Ubiquinone and other terpenoid-quinone biosynthesis',
 'Biotin metabolism',
 'Tryptophan metabolis

In [8]:
pathways_count = {}
for key in pathways:
    aux = df_saqua_ncbi[df_saqua_ncbi["Pathway"].str.contains(key)]
    pathways_count[key] = aux["Pathway"].count()
pathways_count

C:\Users\eulle\Anaconda3\lib\site-packages\pandas\core\strings.py:1843: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


{'Necroptosis': 90,
 'Sphingolipid metabolism': 7,
 'Ether lipid metabolism': 117,
 'Quorum sensing': 897,
 'Glycerolipid metabolism': 164,
 'Biosynthesis of vancomycin group antibiotics': 12,
 'Bacterial secretion system': 337,
 'Tuberculosis': 29,
 'Peroxisome': 138,
 'Renal cell carcinoma': 1,
 'MAPK signaling pathway - yeast': 6,
 'Cushing syndrome': 1,
 'Non-alcoholic fatty liver disease (NAFLD)': 0,
 'DNA replication': 148,
 'Salmonella infection': 92,
 'Viral carcinogenesis': 2,
 'Biotin metabolism': 470,
 'PPAR signaling pathway': 154,
 'Biosynthesis of secondary metabolites': 7023,
 'Axon regeneration': 1,
 'Phosphonate and phosphinate metabolism': 138,
 'Carbapenem biosynthesis': 364,
 'Central carbon metabolism in cancer': 512,
 'Dioxin degradation': 10,
 'Lysine biosynthesis': 417,
 'Plant-pathogen interaction': 112,
 'Thyroid hormone signaling pathway': 117,
 'AMPK signaling pathway': 31,
 'Biofilm formation - Pseudomonas aeruginosa': 409,
 'MAPK signaling pathway - fly': 

In [36]:
# Gerando o plot das vias
# Como o num de vias é muito grande, vou repartir em dois gráficos
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter

pathways_1 = [0:111]
pathways_2 = [111:221]

x = pd.DataFrame(pathways_count.items(),columns=["Pathway","Count"],index=list(pathways_count.keys()))
x = x.sort_values(by=["Count"],ascending=False)
ax = x.plot.barh(figsize=(500, 200), color='#86bf91', zorder=2, width=0.95)
ax.set_alpha(0.8)
ax.set_title("Saqua x NCBI", fontsize=15)
ax.set_xlabel("Count of reads per pathway", fontsize=15);
ax.set_xticks([0,500,1300,2000,3000,4200])

for i in ax.patches:  
    ax.text(i.get_width()+.1, i.get_y()+.60, \
            str(round((i.get_width()), 2)), fontsize=10, color='dimgrey')

# invert for largest on top 
ax.invert_yaxis()


NameError: name 'pathways_count' is not defined

In [1]:
# Verificar quais são as vias que foram mapeadas tanto no NCBI quanto no Biosurfdb
# As vias do ncbi foram geradas em uma célula anterior
df_saqua_biosurfdb = pd.read_table("saqua_biosurfdb_final.csv")
df_saqua_biosurfdb = df_saqua_biosurfdb[df_saqua_biosurfdb["Pathway"].notna()]
pathways_biosufdb = df_saqua_biosurfdb["Pathway"]
pathways_biosufdb = list(pathways_biosufdb)
pathways_biosufdb = map(methodcaller("split",";"),pathways_biosufdb)
pathways_biosufdb = functools.reduce(operator.iconcat, pathways_biosufdb, [])
pathways_biosufdb = list(set(pathways_biosufdb))

bs_bd = list(set(pathways).intersection(pathways_biosufdb))

set(pathways_biosufdb) - set(bs_bd)

NameError: name 'pd' is not defined

In [40]:
pathways[0:51]

['Carbapenem biosynthesis',
 'Geraniol degradation',
 'Fluid shear stress and atherosclerosis',
 'Biosynthesis of amino acids',
 'Pathogenic Escherichia coli infection',
 'Valine, leucine and isoleucine biosynthesis',
 'Pertussis',
 'Drug metabolism - cytochrome P450',
 'Dioxin degradation',
 'Proximal tubule bicarbonate reclamation',
 'Tryptophan metabolism',
 'D-Alanine metabolism',
 'Base excision repair',
 'Primary immunodeficiency',
 'Monobactam biosynthesis',
 'Choline metabolism in cancer',
 'Apoptosis - fly',
 'Ribosome biogenesis in eukaryotes',
 'Antifolate resistance',
 'Insulin signaling pathway',
 'Chloroalkane and chloroalkene degradation',
 'Peptidoglycan biosynthesis',
 'Necroptosis',
 'Bacterial chemotaxis',
 'Butanoate metabolism',
 'Epithelial cell signaling in Helicobacter pylori infection',
 'Glycosphingolipid biosynthesis - globo and isoglobo series',
 'Chagas disease (American trypanosomiasis)',
 'Synthesis and degradation of ketone bodies',
 'Glutamatergic synap